## Advanced Lane Finding Project

The goals / steps of this project are the following:

* [Camera calibration undistortion of image](#Calibration)
* [Applying color gradient transforms](#ColorGradient)
* [Perspective transform](#PerspectiveTransform)
* [Detect lane pixels and fit to find the lane boundary](#FindLanePixel)
* [Determine the curvature of the lane and vehicle position with respect to center](#RadiusCurvature)
* [Warp the detected lane boundaries back onto the original image](#BackToOriginalImage)
* [Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position](#PipelineOnVideo)

---



In **P2.ipynb** file you will find a similar table of content such that it is easy to navigate between both notebooks.

## Camera calibration undistortion of image<a name="Calibration"/>


Cameras transform a 3D image in a 2D one.


In addition real cameras use curved lenses to form an image, and light rays often bend a little too much or too little at the edges of these lenses. This creates an effect that distorts the edges of images, so that lanes or objects appear more or less curved than they actually are. This is called radial distortion, and it’s the most common type of distortion.

Another type of distortion, is tangential distortion. This occurs when a camera’s lens is not aligned perfectly parallel to the imaging plane, where the camera film or sensor is. This makes an image look tilted so that some objects appear farther away or closer than they actually are.

![Calibration](output_images/pinholecamera_distortion.png?raw=true "Calibration")

There are three coefficients needed to correct for radial distortion: k1, k2, and k3. To correct the appearance of radially distorted points in an image, one can use a correction formula.

There are two more coefficients that account for tangential distortion: p1 and p2, and this distortion can be corrected using a different correction formula.

![Radial distortion](output_images/formula_radial_distortion.png?raw=true "Radial distortion")
![Tengential distortion](output_images/formula_tengential_distortion.png?raw=true "Tengential distortion")


See **P2.ipynb**, inside you can find camera calibration and parameters being saved to disk such that they must  not be always recomputed.
See `Deliver calibration parameters (if pickle file exists)` and `Compute calibration parameters and save them into a file`

Following image can be reproduced running the cell located below the title `Test calibration`.

The method to be called is `undistort`. An example a an image are provided. Here is a a dump of the image:

![Calibration](output_images/calibration.png?raw=true "Calibration")

## Applying color gradient transforms<a name="ColorGradient"/>

Computer cannot understand an image, for this reason some algorithms need to be applied.

To simplify the usage of such algorithms a pre process is needed. In this case I applied the following operators (See code below the title `Color gradient transform`):

* Extract l and s channels from a HLS converted image

`l_channel, s_channel_thresholded = hls_select(image_orig, hls_thresh)`
* Compute sobel (derivative gradient) in x and y
    
`sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)`

`sobely = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)`     

* Compute a thresholded x gradient image

`image0 = abs_sobel_threshold(sobelx, s_channel_thresholded, abs_thresh)`

* Compute a magnitude gradient using x and y sobel operators

`image2 = mag_threshold(sobelx, sobely, s_channel_thresholded, mag_thresh)`

* Computed a directional gradient

`image3 = dir_threshold(sobelx, sobely, s_channel_thresholded, dir_thresh)`

* Combine computed images as follow:

`combined[(image0 == 1) | ((image2 == 1) & (image3 == 1))| (s_channel_thresholded == 1)] = 1`


Below title `Analyze best values for gradients`, the cell is able to compute many images varying many parameters.
I played with many combinations to extract the values that fit the best.

However later on when it came to perspective transform I realized that the optimizations I had looked for were not helping: 
* the shadows or black lanes on the road of the video were the most annoying problem.
* unfortunately none of the original picture were having such a problem  

Here are some examples of such images with parameters before they were tuned (some more values might need to be added in the for loops to reproduce them):

![test5-ksize_7-abs_30-mag_30-angle_50-col_90_110](output_images/test5-ksize_7-abs_30-mag_30-angle_50-col_90_110.png?raw=true "test5-ksize_7-abs_30-mag_30-angle_50-col_90_110")

![test5-ksize_7-abs_45-mag_15-angle_50-col_90_110](output_images/test5-ksize_7-abs_45-mag_15-angle_50-col_90_110.png?raw=true "test5-ksize_7-abs_30-mag_30-angle_50-col_90_110")



Below `Test best values of gradient color transform over all images` there is an example of which best values fit:
        
| kSize  | abs threshold | mag threshold |   angle    | color threshold |        
| ------ | ------------- | ------------- |------------|-----------------|         
|   9    |   30, 225     |   30, 225     | (0, pi/2)  |    (0, 165)     |         


![test6-ksize_9-abs_30-mag_30-angle_90-col_90-final](output_images/test6-ksize_9-abs_30-mag_30-angle_90-col_90-final.png?raw=true "test5-ksize_7-abs_30-mag_30-angle_50-col_90_110")

![test5-ksize_9-abs_30-mag_30-angle_90-col_90-final](output_images/test5-ksize_9-abs_30-mag_30-angle_90-col_90-final.png?raw=true "test5-ksize_9-abs_30-mag_30-angle_90-col_90-final")


## Perspective transform<A name = "PerspectiveTransform"/>

We need a so called **top down** or **bird eye** view such that we can apply a polynomial regression on the lanes. For this algorithms we need to define a square both the original image that maps in the transfomed image.

Such a rectangle should be computed but given current image, I extracted values based on image inspection, see below:

#### Points coordinates on the left image
| left top  | right top | left bottom |   right bottom    |     
| ------ | ------------- | ------------- |------------| 
|   574, 463    |   712, 463    |   242, 670     | 1050, 670  |        

#### Points coordinates on the right image
| left top  | right top | left bottom |   right bottom    |     
| ------ | ------------- | ------------- |------------| 
|   100, 10    |   1100, 10    |   1100, 683     | 1100, 683  |        



![perspective_transformed_image](output_images/perspective_transformed_image.png?raw=true "perspective_transformed_image")


See code below `Apply a perspective transform to rectify binary image ("birds-eye view")` and test images below `Test perspective transform`

## Detect lane pixels and fit to find the lane boundary<A type = "D"/>

Once lanes are parallel:
1. We can use a histogram to dicsover where they are located on the image
2. Being parallel does not mean straight and indeed lanes can have a curve, for this reason the histogram mut be applied on sliding windows.
3. We define 9 sliding windows per image, a margin of 100 pixels on both sides and to be recognized as a lane, an histogram must have a value of at least 50 (Others are outliers)
4. The sliding windows are recentered after each step to be sure we follow the curve.
5. The function `find_lane_pixels` delivers a set of values for xleft and xright of the lanes found in the sliding window
6. The function `fit_polynomial`will then perform a polynomial fit of level 2 over each set of values and return 
    * `out_img` used for degug, 
    * `ploty` defining the y space, 
    * `left_fit` defines the 2nd level of polynomial coefficients of the left lane, 
    * `right_fit` defines the 2nd level of polynomial coefficients of the right lane.


In `Test find lane pixel` you can find several examples based on the test images. Here one example provided:

![FindLanePixel](output_images/FindLanePixel.png?raw=true "FindLanePixel")


## Determine the curvature of the lane and vehicle position with respect to center<A name = "RadiusCurvature">


We want to apply the radius of curvature on our polynomial fitted expression. The function `measure_curvature_pixels` takes care of this. It takes as input parameters: 

* `ploty` defining the y space, 
* `left_fit` defines the 2nd level of polynomial coefficients of the left lane, 
* `right_fit` defines the 2nd level of polynomial coefficients of the right lane.
    
and delivers 2 radius of curvatures, one for the left lane and one for the right lane.

In addition the functions transforms the pixel distances into meters: For 720 pixels we consider we cover a distance of 30 meters -> thus we apply a factor of 30 / 270 on the y values (y_eval in the function `measure_curvature_pixels`)

Code regarding this can be found in `Determine the curvature of the lane and vehicle position with respect to center`.
Here are some examples of values:


Image: test_images\straight_lines1.jpg, curvature: 14106.661785 m  -> Lines are straight 
Image: test_images\straight_lines2.jpg, curvature: 13930.313901 m  -> Lines are straight
Image: test_images\test1.jpg, curvature: 1340.649505 m
Image: test_images\test2.jpg, curvature: 404.787499 m
Image: test_images\test3.jpg, curvature: 1500.485561 m
Image: test_images\test4.jpg, curvature: 484.221080 m
Image: test_images\test5.jpg, curvature: 612.917353 m
Image: test_images\test6.jpg, curvature: 3126.172337 m

When lines are straight the curvature gets huge which makes sense.
    
The vehicle position with respect to the center is easily calculated considering the bottom poition of the lines. Say we have pos_x_left and pos_x_right. Center is: 

`(pos_x_right - pos_x_left) / 2 + pos_x_left`

The deviation can be calculated with `(pos_x_right - pos_x_left) / 2 + pos_x_left - image.shape[1] / 2`
    

## Warp the detected lane boundaries back onto the original image <A name = "BackToOriginalImage">

The perspective transform must now be applied back on the image, for this we use an invertied matrix.

See below for some example of images.

See section `Test green path` to reproduce these images.

The following line code:
    
`result = create_green_path(image_bgr, left_line, right_line, True, True)`

Displays all images. If the last parameter of the function is switched to False, all debug information will be removed.
    
A few notes regarding the debug information: These information are mostly relevant when running the video:
    
* The top left image prints details about positioning of lanes, diff on the x axis , possible errors found and curvature in pixels.
    * The curvature is measured at the bottom `image.shape[0]` and as weel at following y positions: 
    * `image.shape[0] / 2` and 
    * `3 * image.shape[0] / 4`
    * The reason for this is that at the bottom it could be perfectly fine but at a different y coord it could be too high or too low.
* The top right image is the binary wrapped image and the polynomial regression
* The left bottom image displays some graphical debug information:
    * Location where curvature is measured
    * Shift of the base position 
* The last image shows the histogram computation and corresponding polynomial regression
   

![green_path_straight_lines2](output_images/green_path_straight_lines2.jpg?raw=true "green_path_straight_lines2")
![green-path-test2](output_images/green-path-test2.png?raw=true "green-path-test2")
![green-path-test6](output_images/green-path-test6.png?raw=true "green-path-test6")


## Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position<A name = 'PipelineOnVideo'>
 

On images all worked fine however once applied on the video it did not work so good anymore.

* Shadows in the video was confusing the algorithm
    * I tried to handle this with some croping but this dis not help.

I then tried to correct the problem analyzing following parameters:
* Radius of curvature
    * The radius of curvature being measured at the bottom was not enough thus I measured at 3 different places (see previous section for more details)
    * If all 3 measures do not restrict the lane it is temporarily accepted
* Center of deviation
    * As shadows are confusing the algorithm making sure to restrict the center of deviation to acceptable values allowed to imrpove a lot
    * Still sometimes shadows are displayed in a such a way that the center of deviation cannot restrict them (too close to the real lanes)
    * This is measured against the lanes that are so far accepted meaning that for each lane:
        * Either new analyzed lane is properly detected and is taken into account for this measure
        * Or previous lane is used
        * In other words, say:
            * Left lane has an unacceptable radius of curvature
            * Right lane is fine
            * Then Center of deviation must be measured between latest accepted left lane and new measured right lane
            * If the center if deviation does not fit right lane is discarded too and we continue with previous lanes
* Distance between lanes
    * If the distance between lanes exceed some threshold both lanes are discarded
    * Again we need to make sure this applies to the lanes about to be displayed (possibly one old lane against one new measured lane)
    
In the left / bottom screen, when:
* a lane is discarded it is displayed in red, 
    * previous accepted lane is displayed in green and bold
* a lane is accepted it is displayed in yellow

The problems I still see are the following:
* Two different shadows are distant in the same proportion as the 2 lanes: This confuses this algorithm
    * I tried to handle this restricting the position of the lanes themselves 
    * Despite this worked quite good on the video the algorithm was overfitted and could not process successfully the images provided.
* The car changes its center of deviation and for arouns 1 second no line can be successfully detected
    * This was mostly due of the impact of the shadows influencing the histogram analyzis such that the radius of curvature became so low that it was not acceptable anymoe.
    * An improvement for this problem would be to be able to get rid of the influznce of the shadows. I tried several combinations but could not come to a good result.
    
A video with all debug information is provided with the name: [video with debug](./out_challenge_video-debug.mp4)

To reproduce the video with debug change the latest boolean from False to True, meaning from:

`modified_image = create_green_path(image, left_line, right_line, False, False)`

to:

`modified_image = create_green_path(image, left_line, right_line, False, True)`


A video without the debug information can be found with the name:  [video without debug](./out_challenge_video.mp4)

    
![video1](output_images/video1.png?raw=true "video1")
![video2](output_images/video2.png?raw=true "video2")


### Discussion

#### Briefly discuss any problems / issues you faced in your implementation of this project. 

* I spent a lot of time tuning many parameters, 
* I had planned to provide some kind of lean based on many accepted values but then realized that considering the few number of detected lines, the road shape would have changed already such making this concept contra productive.
* I tried many tuning in the color gradient as well without the expected big success: This is where I would spend more time if I could.
* Thie pipeline is as well extremely slow and not usable in real time in a car, this could be improved with some efficient machine learning algorithm like convutional deep learning or maybe even XBoost.

